# Tensorflow: классификация изображений MNIST

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

/home/truename/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/truename/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/truename/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/truename/anaconda3/lib/python3.7/site-packages/tensorflow/python/frame

In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

## Функции инициализаии переменных и создания слоев

In [3]:
def flatten(x):
    """ Преобразование тензора в вектор """
    size = 1
    for dimension in x.shape[1:]:
        size *= dimension.value
    return tf.reshape(x, [-1, size])

def fc(x, size):
    """ Полносвязный слой """
    in_size = x.shape[-1].value  # получаем размерность входных данных
    # тензор весов нейронов
    w_shape = [in_size, size]
    w = tf.get_variable(name='w', initializer=tf.truncated_normal(shape=w_shape))
    # тензор свободных членов нейронов
    b_shape = [size]
    b = tf.get_variable(name='b', initializer=tf.truncated_normal(shape=b_shape))
    return tf.matmul(x, w) + b    

def conv2d(x, ksize, filters):
    """ Реализация операции 2D-свертки """
    x_channels = x.shape[-1].value  # число каналов во входном тензоре
    # тензор сверточных фильтров
    w_shape = [ksize, ksize, x_channels, filters]
    w = tf.get_variable(name='w', initializer=tf.truncated_normal(shape=w_shape))
    # свободный член
    b_shape = [filters]
    b = tf.get_variable(name='b', initializer=tf.truncated_normal(shape=b_shape))
    return tf.nn.conv2d(x, filter=w, strides=[1, 1, 1, 1], padding='SAME') + b

## Задаем архитектуру сети

In [4]:
graph = tf.Graph()
optimzer = tf.train.AdamOptimizer(learning_rate=3e-4)

with graph.as_default():
    X = tf.placeholder(tf.float32, shape=[None, 28, 28, 1]) # входные изображения размера 28x28
    Y = tf.placeholder(tf.float32, shape=[None, 10])  # one-hot encoding на 10 классов

    with tf.variable_scope('conv_1'):
        conv_1 = conv2d(X, ksize=5, filters=8)
        conv_1 = tf.nn.relu(conv_1)
        conv_1 = tf.nn.max_pool(conv_1,
                                ksize=[1, 2, 2, 1], 
                                strides=[1, 2, 2, 1],
                                padding='SAME')
        
    with tf.variable_scope('conv_2'):
        conv_2 = conv2d(conv_1, ksize=5, filters=16)
        conv_2 = tf.nn.relu(conv_2)
        conv_2 = tf.nn.max_pool(conv_2,
                                ksize=[1, 2, 2, 1], 
                                strides=[1, 2, 2, 1],
                                padding='SAME')
    
    with tf.variable_scope('fc1'):
        fc1 = flatten(conv_2)
        fc1 = fc(fc1, size=10)

    loss = tf.losses.softmax_cross_entropy(Y, fc1)
    train_step = optimzer.minimize(loss)
    
    prediction = tf.nn.softmax(fc1, name='prediction')
    # TODO: добавить подсчет доли правильных ответов (accuracy)
    
    summary_train = tf.summary.merge([
        tf.summary.scalar('train/loss', loss),])

    summary_test = tf.summary.merge([
        tf.summary.scalar('test/loss', loss),])


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


## Запускаем процесс обучения

In [5]:
!rm -rf ./log/002

In [6]:
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    writer = tf.summary.FileWriter('./log/002', sess.graph)
    
    for i in range(500):
        # загружаем данные для шага обучения
        train_images, train_labels = mnist.train.next_batch(50)
        train_images = train_images.reshape(-1, 28, 28, 1)
        # вычисляем градиенты и обновляем параметры модели
        (_,
         train_loss,
         train_summary) = sess.run([train_step,
                                    loss,
                                    summary_train],
                                   feed_dict={X: train_images,
                                              Y: train_labels})
        # логируем данные для вывода на tensorboard
        writer.add_summary(train_summary, global_step=i)  
        
        if i % 100 == 0:  # оцениваем качество модели на тесте
            test_images = mnist.test.images.reshape(-1, 28, 28, 1)
            (test_loss,
             test_summary) = sess.run([loss,
                                       summary_test],
                                      feed_dict={X: test_images,
                                                 Y: mnist.test.labels})
            # логируем данные для вывода на tensorboard
            writer.add_summary(test_summary, global_step=i)
            print('[%04d] train_loss=%g test_loss=%g' % (i, train_loss, test_loss))
            
        writer.flush()
        
    writer.close()

[0000] train_loss=659.481 test_loss=560.816
[0100] train_loss=194.378 test_loss=193.688
[0200] train_loss=143.228 test_loss=120.598
[0300] train_loss=72.1738 test_loss=99.5792
[0400] train_loss=101.193 test_loss=84.5161


## Визуализация в tensorboard

In [7]:
# запускаем интерфейс визуализации
!tensorboard --port 8800 --host 0.0.0.0 --logdir ./log/002

/home/truename/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/truename/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/truename/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/truename/anaconda3/lib/python3.7/site-packages/tensorflow/python/frame